In [9]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score
import numpy as np

# 데이터 준비 

In [4]:
cancer = load_breast_cancer()

In [5]:
type(cancer)

sklearn.utils._bunch.Bunch

In [6]:
cancer["data"].shape

(569, 30)

In [7]:
cancer["data"][0]

array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
       3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
       8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
       3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
       1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01])

In [8]:
cancer["target"][:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
np.unique(cancer["target"], return_counts = True)

(array([0, 1]), array([212, 357], dtype=int64))

- 종속변수(target)
    - 0 : 악성
    - 1 : 양성

In [13]:
# 컬럼명
cancer["feature_names"], len(cancer["feature_names"])

(array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
        'mean smoothness', 'mean compactness', 'mean concavity',
        'mean concave points', 'mean symmetry', 'mean fractal dimension',
        'radius error', 'texture error', 'perimeter error', 'area error',
        'smoothness error', 'compactness error', 'concavity error',
        'concave points error', 'symmetry error',
        'fractal dimension error', 'worst radius', 'worst texture',
        'worst perimeter', 'worst area', 'worst smoothness',
        'worst compactness', 'worst concavity', 'worst concave points',
        'worst symmetry', 'worst fractal dimension'], dtype='<U23'),
 30)

In [14]:
cancer["target_names"]

array(['malignant', 'benign'], dtype='<U9')

# 데이터 분할

In [21]:
x_train, x_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                   test_size = 0.25, stratify = cancer.target,
                                                   random_state = 11)

In [22]:
len(x_train), len(x_test)

(426, 143)

- cancer.data :독립변수
- cancer.target : 종속변수

# 모델 학습 

In [23]:
clf = SVC()
clf.fit(x_train, y_train)

SVC()

In [24]:
pred = clf.predict(x_test)

In [26]:
accuracy_score(y_test, pred)

0.9440559440559441

# 최적화

In [27]:
mm = MinMaxScaler()
scaled_train = mm.fit_transform(x_train)
scaled_test = mm.transform(x_test)

In [28]:
clf2 = SVC()
clf2.fit(scaled_train, y_train)

SVC()

In [29]:
clf2.score(scaled_test, y_test)

0.958041958041958

# 하이퍼파라미터 튜닝

- 머신러닝 모형이 완성되고나서 매개변수 최적화를 통해서 예측성능을 더 극대화

In [32]:
param_grid = {"C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
            "gamma" : [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [33]:
grid = GridSearchCV(SVC(random_state = 11), param_grid = param_grid, cv = 5, n_jobs = -1)

In [34]:
grid.fit(scaled_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=11), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [35]:
grid.best_score_

0.9812311901504789

In [36]:
grid.best_params_

{'C': 1, 'gamma': 1}

In [37]:
grid.score(scaled_test, y_test)

0.951048951048951

# 파이프라인 구축

- 여러 알고리즘을 비교
- 전체 공정을 시퀀스로 연결하여 구성

## 파이프라인 구축 예

- case1 : 표준 방법
    - 작업 순서를 나열하고, 이름을 각각 부여
    - 후보가 유력한 알고리즘과 전처리기를 배치

In [39]:
pipe = Pipeline([("scaler", MinMaxScaler()),
                ("classifer", SVC(random_state = 11))])

- scaler,classifier 이 두개는 내가 이름 임의로 지정

In [40]:
pipe

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('classifer', SVC(random_state=11))])

- case2 : 간소화 방법
    - 이름을 부여하지 않고 자동으로 부여됨

In [41]:
pipe_sim = make_pipeline(MinMaxScaler(), SVC())
pipe_sim

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC())])

# 파이프라인 적용

In [44]:
pipe = Pipeline([("preprocessing", StandardScaler()),
                ("classifier", SVC(random_state = 11))])

In [45]:
pipe

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('classifier', SVC(random_state=11))])

In [51]:
# 하이퍼 파라미터 정의
param_grid = [
    {
        "classifier" : [SVC(random_state = 11)],
        "preprocessing" : [StandardScaler(), MinMaxScaler()],
        # 이름__ 파라미터명 => 해당 알고리즘에 적용되는 파라미터
        "classifier__C" : [0.001, 0.01, 0.1, 1, 10, 100],
        "classifier__gamma" : [0.001, 0.01, 0.1, 1, 10, 100]
    },
    {
        "classifier" : [RandomForestClassifier(random_state = 11)],
        "preprocessing" : [None],
        "classifier__max_features" : [1, 2, 3]
    }
]

In [52]:
# 결합(파이프라인, 하이퍼파라미터 튜닝)
grid = GridSearchCV(pipe, param_grid, cv = 5, n_jobs = -1)

In [53]:
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', StandardScaler()),
                                       ('classifier', SVC(random_state=11))]),
             n_jobs=-1,
             param_grid=[{'classifier': [SVC(C=10, gamma=0.1, random_state=11)],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'preprocessing': [StandardScaler(), MinMaxScaler()]},
                         {'classifier': [RandomForestClassifier(random_state=11)],
                          'classifier__max_features': [1, 2, 3],
                          'preprocessing': [None]}])

In [54]:
grid.best_score_

0.9812311901504789

In [55]:
grid.best_params_

{'classifier': SVC(C=10, gamma=0.1, random_state=11),
 'classifier__C': 10,
 'classifier__gamma': 0.1,
 'preprocessing': MinMaxScaler()}

In [56]:
grid.score(x_test, y_test)

0.958041958041958